# Glider Speed polars analysis notebook

In [356]:
import numpy as np
from numpy.polynomial import Polynomial

print(np.__version__)

# https://www.geeksforgeeks.org/python-implementation-of-polynomial-regression/?ref=lbp
# old vs new API https://numpy.org/doc/1.21/reference/routines.polynomials.html

1.21.1


In [357]:
# utilities functions & constants

KM_TO_MS = 3.6		# factor to convert km/h in m/s
CONVERT_TO_MS = True

POLAR_CURVE_START_KM = 70		# in km/h
POLAR_CURVE_END_KM = 230		# in km/h

POLAR_CURVE_START = (POLAR_CURVE_START_KM / KM_TO_MS) if CONVERT_TO_MS else POLAR_CURVE_START_KM
POLAR_CURVE_END = (POLAR_CURVE_END_KM / KM_TO_MS)if CONVERT_TO_MS else POLAR_CURVE_END_KM

#POLAR_CURVE_NBR_SAMPLE = round((POLAR_CURVE_END - POLAR_CURVE_START) // 10)
POLAR_CURVE_NBR_SAMPLE =  20

def polar_to_string(p):
	polar_str = '{}x<sup>2</sup>'.format(round(p.coeffs[0],4))
	polar_str += (' + ' if (p.coeffs[1] >= 0 ) else ' ')
	polar_str += '{}x'.format(round(p.coeffs[1],4))
	polar_str += (' + ' if (p.coeffs[2] >= 0 ) else ' ')
	polar_str += '{}'.format(round(p.coeffs[2],4))
	return polar_str

xaxis_unit = lambda x: x/ (KM_TO_MS if CONVERT_TO_MS else 1)

In [358]:

## Polar data for LAK-19 18m
import json
glider_pdata = None	# glider polars data

# Opening JSON file
with open('./lak19-polars-db.json') as json_file:
	glider_pdata = json.load(json_file)

seeyouABC_data = glider_pdata['specifications']['Naviter SeeYou-2']
manual_data = glider_pdata['specifications']['From Manual']
iglide_data = glider_pdata['specifications']['iGlide']
seeyou_data = glider_pdata['specifications']['Naviter SeeYou-1']

In [359]:

# Data for the LAK-19 18m (from polar on the manual), wing loading 35.7 kg/m2
# c1 curve will be the calculated polar using the 3 points method, 
# the 3 points are estimated based on the flight polar diagram of the manual, page 5.4

c1_x = [xaxis_unit(x) for x in manual_data['speed']]
#x = [100/3.6,120/3.6,150/3.6]
c1_y = manual_data['sink_rate']
c1_wl = manual_data['wing_loading']

# New APIs
# polar = Polynomial.fit(x, y, 2) 
# print(' polynom with new API is {}'.format(polar))
# print(' coef. is {}, domain is {}, window is {}'.format(polar.coef,polar.domain, polar.window))
# print(polar.convert())

# Legacy APIs
c1_polar = np.poly1d(np.polyfit(c1_x, c1_y, 2))
print ('Polynom with legacy API is')
print(c1_polar) 

c1_x_polar = np.linspace(POLAR_CURVE_START, POLAR_CURVE_END, POLAR_CURVE_NBR_SAMPLE)
c1_y_polar = c1_polar(c1_x_polar)

print('\r\nc1_y_polar = {}'.format(c1_y_polar))

Polynom with legacy API is
           2
-0.002808 x + 0.1446 x - 2.45

c1_y_polar = [-0.7        -0.63255771 -0.59584488 -0.5898615  -0.61460757 -0.6700831
 -0.75628809 -0.87322253 -1.02088643 -1.19927978 -1.40840259 -1.64825485
 -1.91883657 -2.22014774 -2.55218837 -2.91495845 -3.30845799 -3.73268698
 -4.18764543 -4.67333333]


In [360]:
# To plot Naviter Seeyou polar y = - (A * x^2 + B * x + C)

#c2_polar = np.poly1d([seeyou_data["A"]*-1,seeyou_data["B"]*-1, seeyou_data["C"]*-1 ])
c2_polar = np.poly1d([seeyouABC_data["A"],seeyouABC_data["B"], seeyouABC_data["C"] ])

print ('Polynom with legacy API is')
print(c2_polar) 

c2_x_polar = np.linspace(POLAR_CURVE_START, POLAR_CURVE_END, POLAR_CURVE_NBR_SAMPLE)
c2_y_polar = c2_polar(c2_x_polar)

print('\r\nc2_y_polar = {}'.format(c2_y_polar))

Polynom with legacy API is
      2
1.24 x - 1.61 x + 0.97

c2_y_polar = [ 438.49160494  554.31103656  683.70045552  826.65986184  983.1892555
 1153.2886365  1336.95800486 1534.19736056 1745.0067036  1969.38603399
 2207.33535173 2458.85465682 2723.94394925 3002.60322903 3294.83249615
 3600.63175062 3920.00099244 4252.94022161 4599.44943812 4959.52864198]


In [361]:
# To plot iGlide polar (3 points methods), 

# Fit a polynomial of degree 2 based on the 3 points iglide data
# The c3 curve will be the calculated polar using this polynomial, 

c3_x = [xaxis_unit(x) for x in iglide_data['speed']]
c3_y = iglide_data['sink_rate']
c3_wl = iglide_data['wing_loading']

# rgression polynomial
c3_polar = np.poly1d(np.polyfit(c3_x, c3_y, 2))
print ('Polynomial model is')
print(c3_polar) 

c3_x_polar = np.linspace(POLAR_CURVE_START, POLAR_CURVE_END, POLAR_CURVE_NBR_SAMPLE)
c3_y_polar = c3_polar(c3_x_polar)

print('\r\nc3_y_polar = {}'.format(c3_y_polar))

Polynomial model is
           2
-0.002621 x + 0.1323 x - 2.133

c3_y_polar = [-0.55206231 -0.495398   -0.46741259 -0.46810606 -0.49747842 -0.55552967
 -0.64225981 -0.75766884 -0.90175676 -1.07452356 -1.27596926 -1.50609384
 -1.76489731 -2.05237967 -2.36854092 -2.71338106 -3.08690008 -3.489098
 -3.9199748  -4.3795305 ]


In [362]:
# To plot Seeyou polar (3 points methods), 

# Fit a polynomial of degree 2 based on the 3 points SeeYou data
# The c4 curve will be the calculated polar using this polynomial, 

c4_x = [xaxis_unit(x) for x in seeyou_data['speed']]
c4_y = seeyou_data['sink_rate']
c4_wl = seeyou_data['wing_loading']

# rgression polynomial
c4_polar = np.poly1d(np.polyfit(c4_x, c4_y, 2))
print ('Polynomial model is')
print(c4_polar) 

c4_x_polar = np.linspace(POLAR_CURVE_START, POLAR_CURVE_END, POLAR_CURVE_NBR_SAMPLE)
c4_y_polar = c4_polar(c4_x_polar)

print('\r\nc3_y_polar = {}'.format(c4_y_polar))

Polynomial model is
           2
-0.001642 x + 0.06072 x - 1.02

c3_y_polar = [-0.46       -0.4762807  -0.51052632 -0.56273684 -0.63291228 -0.72105263
 -0.82715789 -0.95122807 -1.09326316 -1.25326316 -1.43122807 -1.62715789
 -1.84105263 -2.07291228 -2.32273684 -2.59052632 -2.8762807  -3.18
 -3.50168421 -3.84133333]


In [363]:
# Display the graph
# https://plotly.com/python/plotly-fundamentals/
# https://plotly.com/python-api-reference/

import plotly.graph_objects as go

trace1 = go.Scatter( x=c1_x, y=c1_y, mode='markers', name='A, B, C points', 
	text=['Point A', 'Point B', 'Point C'], hovertemplate='<extra></extra><b>%{text}</b><br>Speed: %{x}km/h<br>Sink rate: %{y}m/s', hoverinfo='x+y+text')
	
trace2 = go.Scatter( x=c1_x_polar, y=c1_y_polar, mode='lines', name='Manual (wing loading: {} kg/m<sup>2</sup>)'.format(c1_wl),
		hovertemplate='<extra></extra>Speed: %{x:.0f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y')

trace3 = go.Scatter( x=c2_x_polar, y=c2_y_polar, mode='lines', name='SeeYou ABC (wing loading: {} kg/m<sup>2</sup>)'.format(c1_wl),
		hovertemplate='<extra></extra>Speed: %{x:.0f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y')

trace4 = go.Scatter( x=c3_x_polar, y=c3_y_polar, mode='lines', name='iGlide (wing loading: {} kg/m<sup>2</sup>)'.format(c3_wl), line = dict(dash='dash'),
		hovertemplate='<extra></extra>Speed: %{x:.0f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y')

trace5 = go.Scatter( x=c4_x_polar, y=c4_y_polar, mode='lines', name='SeeYou (wing loading: {} kg/m<sup>2</sup>)'.format(c4_wl), line = dict(dash='dash'),
		hovertemplate='<extra></extra>Speed: %{x:.0f}km/h<br>Sink rate: %{y:.2f}m/s', hoverinfo='x+y')

layout = go.Layout(
		title_text='<b>{}  - Speed polars analysis</b>'.format(glider_pdata['glider']),
		height=600,
		yaxis=dict(range=[-4, 1]),
		template='ggplot2',
	)

data = [trace1, trace2, trace3, trace4, trace5]
fig = go.Figure(data=data, layout=layout)

fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black',  dtick=0.5)
fig.update_yaxes(title_text='Vz (m/s)', title_font=dict(size=14, family='Courier', color='crimson'))

fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black')
fig.update_xaxes(title_text='Vitesse ({})'.format(( 'm/s' if CONVERT_TO_MS else 'km/h')),
	 title_font=dict(size=14, family='Courier', color='crimson'))

fig.add_annotation(x=xaxis_unit(50), y=-2.75, text='Polar: {}'.format( polar_to_string(c1_polar)), showarrow=False)

fig.update_layout(xaxis_range=[0,max(c1_x_polar)*1.2])

fig.show()